In [1]:
import numpy as np
import math
import numpy.linalg as linalg
import pandas as pd
from matplotlib import pyplot
import sklearn as skl
from sklearn.preprocessing import PolynomialFeatures
import heapq


In [2]:
def tricubic(x):
    y = np.zeros_like(x)
    idx = (x >= 0) & (x <= 1)
    y[idx] = np.power(1.0 - np.power(np.abs(x[idx]), 3), 3)
    return y

In [3]:
def normalize(array):
    stds = array.std(axis=0)
    
    return (array- array.mean(axis=0))/stds

In [4]:
def get_indexes(distances, q):
    mins = []
    for i in range(0, len(distances)):
        if len(mins) >= q:
            if mins[0][0] < -1*distances[i]:
                heapq.heapreplace(mins, (-1*distances[i], i))
        else:
            heapq.heappush(mins, (-1*distances[i], i))

    indexes = []
    max_dist = 0.0
    for (dist, index) in mins:
        indexes.append(index)
        if -1*dist > max_dist:
            max_dist = -1*dist
    return indexes, max_dist

In [5]:
def get_weights(distances, max_dist):
    weights = tricubic(distances/max_dist)
    return np.diag(weights)

In [67]:
def loess(data, Y, f=0.1, fit='quadratic', ord=2):
    cant = len(Y)
    q = int(math.ceil(f*cant))
    data_norm = normalize(data)
    if fit == 'quadratic':
        poly = PolynomialFeatures(2)
    
    y_ests = np.zeros_like(Y)
    
    for i in range(0, cant):
        distances = linalg.norm(data_norm - data_norm[i], ord=ord, axis=1)
        indexes, max_dist = get_indexes(distances, q)
        W = get_weights(distances[indexes], max_dist)
        b = Y[indexes]
        if fit=='quadratic':
            A = poly.fit_transform(data_norm[indexes])
        else:
            A = np.append(np.ones((len(indexes),1)), data_norm[indexes], axis=1)
            
        At = np.transpose(A)
        try:
            coeffs = linalg.solve(np.dot(At, np.dot(W, A)), np.dot(At, np.dot(W, b)))
        except:

            U, E, Vt = linalg.svd(np.dot(W, A))
            V = np.transpose(Vt)
            Ut = np.transpose(U)
            
            Et = np.zeros((np.shape(V)[1], np.shape(Ut)[0]))
            
            for j in range(0, len(E)):
                if E[j] != 0:
                    Et[j,j] = 1/E[j]
            
            MPInv = np.dot(V, np.dot(Et, Ut))
            coeffs = np.dot(MPInv, np.dot(W, b))
        
        if fit=='quadratic':
            
            y_est = np.dot(poly.fit_transform(data_norm[i].reshape(1, -1)),coeffs)
        else:
            y_est = np.dot(np.append([1.0], data_norm[i]), coeffs)
        
        y_ests[i] = y_est
    
    return y_ests

In [68]:
data = pd.read_csv('../data/dataset.csv',usecols=['ozone', 'radiation', 'temperature', 'wind'])

In [69]:
x=data[['radiation', 'temperature', 'wind']].to_numpy()
y = data['ozone'].to_numpy()
estimates = loess(x, y, 0.06, 'quadratic', 2)
print(estimates)
print(y)

[ 41  36  11  17  23  18   8  16  10  14  17  14  33   5  30  11   0  10
   4  31  22  44 115  37  29  71  38  23  21  36  19  11  13 134  48  31
  63  40  77  97  96  85   9  26   6  48  35  60  79  62  15  80 108  19
  51  81  49  63  58  38   9  15 121  89 110  44  28  65  21  59  22  31
  44  20   9  44 167  72  76 118  84  85  96  77  73  91  46  31  20  23
  21  24  43  21  27   9  12  45  18  12  23  16  12  22  36   7  13  30
  14  18  20]
[ 41  36  12  18  23  19   8  16  11  14  18  14  34   6  30  11   1  11
   4  32  23  45 115  37  29  71  39  23  21  37  20  12  13 135  49  32
  64  40  77  97  97  85  10  27   7  48  35  61  79  63  16  80 108  20
  52  82  50  64  59  39   9  16 122  89 110  44  28  65  22  59  23  31
  44  21   9  45 168  73  76 118  84  85  96  78  73  91  47  32  20  23
  21  24  44  21  28   9  13  46  18  13  24  16  13  23  36   7  14  30
  14  18  20]
